# Welcome the challenge notebook
---


In this challenge, you will work with a dataset provided by an HR manager who wants to predict which employees are at risk of leaving the company. The dataset contains four key performance indicators (KPIs) related to each employee. Your task is to use PySpark to build a machine learning model that can predict employee attrition and to identify which KPI is most strongly associated with attrition in this company.

- Please note that the dataset is already clean and ready to be modeled.
- The dataset only contains numerical features.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing pyspark

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=454fd29ad1049b1e64869c2aefd089cccdcafb83aace1188d3d2b02d298186a3
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Importing the needed modules and creating the spark session

In [2]:
# importing spark session
from pyspark.sql import SparkSession

# data visualization modules
import matplotlib.pyplot as plt
import plotly.express as px

# pandas module
import pandas as pd

# pyspark data preprocessing modules
from pyspark.ml.feature import  VectorAssembler, StandardScaler,StringIndexer

# pyspark data modeling and model evaluation modules
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# creating the spark session
spark = SparkSession.builder.appName("Challenge").getOrCreate()
spark

Loading the `Challenge_dataset.csv` file

In [5]:
data = spark.read.format('csv').option('header',True).option('inferSchema',True).load('/content/drive/MyDrive/code/coursera projects/Machine Learning with PySpark: Customer Churn Analysis/data/Challenge_dataset.csv')
data.show(5)

+----------+------------------+-------------------+-------------------+-------------------+---------------+
|EmployeeID|              KPI1|               KPI2|               KPI3|               KPI4|CurrentEmployee|
+----------+------------------+-------------------+-------------------+-------------------+---------------+
|         0|1.4347155493478079| 0.8445778971189396| 1.2907117554310856|-1.4201273531837943|              1|
|         1|0.8916245735832885| 0.8308158727699302| 1.0779750584283363|-1.0598957663940176|              1|
|         2|-0.891158353098296|-0.9469681237741348|-1.1825287909456643| 1.1269205082112577|              0|
|         3|1.2797294893867808| 1.6690888870054317| 1.9769417044649022| -1.797525912345404|              1|
|         4|0.2576789316661615|0.34201906896710577|0.40342208520171396|-0.3653830886145554|              1|
+----------+------------------+-------------------+-------------------+-------------------+---------------+
only showing top 5 rows



Create the numerical feature vector using `Vector Assembler`.

Hint: The numerical input features are the KPIs.

In [6]:
assembler = VectorAssembler(inputCols=['KPI1','KPI2','KPI3','KPI4'], outputCol='features')
data_transformed = assembler.transform(data)
data_transformed.show(5)


+----------+------------------+-------------------+-------------------+-------------------+---------------+--------------------+
|EmployeeID|              KPI1|               KPI2|               KPI3|               KPI4|CurrentEmployee|            features|
+----------+------------------+-------------------+-------------------+-------------------+---------------+--------------------+
|         0|1.4347155493478079| 0.8445778971189396| 1.2907117554310856|-1.4201273531837943|              1|[1.43471554934780...|
|         1|0.8916245735832885| 0.8308158727699302| 1.0779750584283363|-1.0598957663940176|              1|[0.89162457358328...|
|         2|-0.891158353098296|-0.9469681237741348|-1.1825287909456643| 1.1269205082112577|              0|[-0.8911583530982...|
|         3|1.2797294893867808| 1.6690888870054317| 1.9769417044649022| -1.797525912345404|              1|[1.27972948938678...|
|         4|0.2576789316661615|0.34201906896710577|0.40342208520171396|-0.3653830886145554|      

Apply `Standard Scaler` to the numerical feature vector

In [7]:
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
data_transformed = scaler.fit(data_transformed).transform(data_transformed)
data_transformed.show(5)


+----------+------------------+-------------------+-------------------+-------------------+---------------+--------------------+--------------------+
|EmployeeID|              KPI1|               KPI2|               KPI3|               KPI4|CurrentEmployee|            features|     scaled_features|
+----------+------------------+-------------------+-------------------+-------------------+---------------+--------------------+--------------------+
|         0|1.4347155493478079| 0.8445778971189396| 1.2907117554310856|-1.4201273531837943|              1|[1.43471554934780...|[1.08046317968569...|
|         1|0.8916245735832885| 0.8308158727699302| 1.0779750584283363|-1.0598957663940176|              1|[0.89162457358328...|[0.67146935313946...|
|         2|-0.891158353098296|-0.9469681237741348|-1.1825287909456643| 1.1269205082112577|              0|[-0.8911583530982...|[-0.6711182493489...|
|         3|1.2797294893867808| 1.6690888870054317| 1.9769417044649022| -1.797525912345404|         

Split the data into train and test sets

In [8]:


train, test = data_transformed.randomSplit([0.7, 0.3])


Train your Decision Tree model

---



In [10]:


dt = DecisionTreeClassifier(featuresCol='scaled_features', labelCol='CurrentEmployee')
dt_model = dt.fit(train)


Perform the prediction on the test set and calculate the accuracy using `BinaryClassificationEvaluator`

In [13]:
predictions = dt_model.transform(test)
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='CurrentEmployee',metricName = "areaUnderROC" )
auc = evaluator.evaluate(predictions)
print("auc of the model:", accuracy)


auc of the model: 0.9186423843011502


Apply the hyper paramter tuning to find the proper `maxDepth` for your decision tree from the `candidates` list.

In [19]:
def evaluate_dt(mode_params):
      test_accuracies = []
      train_accuracies = []

      for maxD in mode_params:
        # train the model based on the maxD
        decision_tree = DecisionTreeClassifier(featuresCol = 'scaled_features', labelCol = 'CurrentEmployee', maxDepth = maxD)
        dtModel = decision_tree.fit(train)

        # calculating test error
        predictions_test = dtModel.transform(test)
        evaluator = BinaryClassificationEvaluator(labelCol='CurrentEmployee')
        auc_test = evaluator.evaluate(predictions_test, {evaluator.metricName: "areaUnderROC"})
        # recording the accuracy
        test_accuracies.append(auc_test)

        # calculating training error
        predictions_training = dtModel.transform(train)
        evaluator = BinaryClassificationEvaluator(labelCol='CurrentEmployee')
        auc_training = evaluator.evaluate(predictions_training, {evaluator.metricName: "areaUnderROC"})
        train_accuracies.append(auc_training)

      return(test_accuracies, train_accuracies)



candidates = list(range(1, 20))
test_auc, train_auc = evaluate_dt(candidates)



Use a line chart to visualize the training and testing accuracy. <br>

Hint: To visualize your data, convert the PySpark dataframe to pandas dataframe.

In [20]:
import pandas as pd
# Convert the PySpark dataframe to pandas dataframe
df_result = pd.DataFrame({'max_depth': candidates, 'test_auc': test_auc, 'train_auc': train_auc})

# Create a line chart to visualize the training and testing auc
px.line(df_result, x='max_depth', y=['test_auc', 'train_auc'], title='Decision Tree Model AUC')


Train the decision tree using the proper `maxDepth` parameter.  

In [21]:
max_depth = 5  # Choose the best maxDepth based on the line chart
dt = DecisionTreeClassifier(featuresCol='scaled_features', labelCol='CurrentEmployee', maxDepth=max_depth)
dt_model = dt.fit(train)


Use the `Feature Importance` to find the most important factor for the employee attrition using a barchart.

In [28]:
importance = dt_model.featureImportances
features = data_transformed.columns[1:-3]
df_importance = pd.DataFrame({'feature': features, 'importance': [importance for importance in dt_model.featureImportances]})
df_importance.sort_values('importance', ascending=False, inplace=True)

# Create a bar chart to visualize the feature importance
px.bar(df_importance, x='feature', y='importance', title='Feature Importance for Employee Attrition')


In [31]:
px.box(data.toPandas(), x='CurrentEmployee', y='KPI4', title='Boxplot of KPI4 vs CurrentEmployee')
